In [1]:
import numpy as np

In [38]:
f = open('test.txt')
lines = f.read()+'\n'

In [39]:
lines

'--- scanner 0 ---\n404,-588,-901\n528,-643,409\n-838,591,734\n390,-675,-793\n-537,-823,-458\n-485,-357,347\n-345,-311,381\n-661,-816,-575\n-876,649,763\n-618,-824,-621\n553,345,-567\n474,580,667\n-447,-329,318\n-584,868,-557\n544,-627,-890\n564,392,-477\n455,729,728\n-892,524,684\n-689,845,-530\n423,-701,434\n7,-33,-71\n630,319,-379\n443,580,662\n-789,900,-551\n459,-707,401\n\n--- scanner 1 ---\n686,422,578\n605,423,415\n515,917,-361\n-336,658,858\n95,138,22\n-476,619,847\n-340,-569,-846\n567,-361,727\n-460,603,-452\n669,-402,600\n729,430,532\n-500,-761,534\n-322,571,750\n-466,-666,-811\n-429,-592,574\n-355,545,-477\n703,-491,-529\n-328,-685,520\n413,935,-424\n-391,539,-444\n586,-435,557\n-364,-763,-893\n807,-499,-711\n755,-354,-619\n553,889,-390\n\n--- scanner 2 ---\n649,640,665\n682,-795,504\n-784,533,-524\n-644,584,-595\n-588,-843,648\n-30,6,44\n-674,560,763\n500,723,-460\n609,671,-379\n-555,-800,653\n-675,-892,-343\n697,-426,-610\n578,704,681\n493,664,-388\n-671,-858,530\n-667,343

In [51]:
sensors = [[list(map(int,x.split(','))) for x in l.split('\n')[1:-1]] for l in lines.replace('\n\n','\n').split('--- scanner')][1:]

In [55]:
sen = dict()
for s,sensor in enumerate(sensors):
    sen[s] = np.asarray(sensor)

In [59]:
sen[0].min(axis=0)
sen[0].max(axis=0)

array([630, 900, 763])

In [62]:
m=sen[1]
m

array([[ 686,  422,  578],
       [ 605,  423,  415],
       [ 515,  917, -361],
       [-336,  658,  858],
       [  95,  138,   22],
       [-476,  619,  847],
       [-340, -569, -846],
       [ 567, -361,  727],
       [-460,  603, -452],
       [ 669, -402,  600],
       [ 729,  430,  532],
       [-500, -761,  534],
       [-322,  571,  750],
       [-466, -666, -811],
       [-429, -592,  574],
       [-355,  545, -477],
       [ 703, -491, -529],
       [-328, -685,  520],
       [ 413,  935, -424],
       [-391,  539, -444],
       [ 586, -435,  557],
       [-364, -763, -893],
       [ 807, -499, -711],
       [ 755, -354, -619],
       [ 553,  889, -390]])

In [64]:
m[:,1],m[:,0],m[:,2]

(array([ 422,  423,  917,  658,  138,  619, -569, -361,  603, -402,  430,
        -761,  571, -666, -592,  545, -491, -685,  935,  539, -435, -763,
        -499, -354,  889]),
 array([ 686,  605,  515, -336,   95, -476, -340,  567, -460,  669,  729,
        -500, -322, -466, -429, -355,  703, -328,  413, -391,  586, -364,
         807,  755,  553]),
 array([ 578,  415, -361,  858,   22,  847, -846,  727, -452,  600,  532,
         534,  750, -811,  574, -477, -529,  520, -424, -444,  557, -893,
        -711, -619, -390]))

In [72]:
#some of these are the same of of each other. So we can do a set and reduce it later if needed.
# can only flip one axes at a time. 
dir = []
for facing in [0,90,180,270]:
    for z in [-1,+1]:
        for y in [-1,+1]:
            for x in [-1,+1]:
                
                dir.append((x,y,z,facing))

In [88]:
def transform(x):
    """Generate all possible rotation functions"""
    vectors = [
        (1, 0, 0),
        (-1, 0, 0),
        (0, 1, 0),
        (0, -1, 0),
        (0, 0, 1),
        (0, 0, -1),
    ]
    vectors = list(map(np.array, vectors))
    r=[]
    for vi in vectors:
        for vj in vectors:
            if vi.dot(vj) == 0:
                vk = np.cross(vi, vj)
                r.append(np.matmul(x, np.array([vi, vj, vk])))
    return r

assert len(transform(sen[1])) == 24

In [109]:
from collections import Counter
mycounter = Counter()
def distance_between_beacons(x,counter):
    
    for i,b in enumerate(x):
        for j,c in enumerate(x):
            if i!=j:
                ans = np.linalg.norm(b-c)
                counter[ans] += 1
    return counter

for s,sensor in enumerate(sensors):
    distance_between_beacons(np.asarray(sensor),mycounter)
    
mycounter.most_common()

[(122.53162856993292, 8),
 (94.45104552094699, 8),
 (49.20365840057018, 8),
 (162.81891781976688, 8),
 (201.0497450881249, 8),
 (162.9018109168833, 8),
 (1054.4235391909647, 6),
 (934.4180006827779, 6),
 (1028.3491624929734, 6),
 (151.2349166032765, 6),
 (55.39855593785816, 6),
 (974.8687091090779, 6),
 (1008.2420344341928, 6),
 (121.2311841070605, 6),
 (862.9791422740182, 6),
 (896.4552414928478, 6),
 (953.2785532046759, 6),
 (985.1949045747242, 6),
 (1242.2640621059597, 6),
 (1173.6191886638528, 6),
 (1092.8961524316937, 6),
 (1179.7749785446376, 6),
 (1157.0367323469036, 6),
 (1138.7620471371533, 6),
 (1318.8168940379858, 6),
 (1438.628861103516, 6),
 (1261.1098286826568, 6),
 (1215.199572086824, 6),
 (1261.8791542774609, 6),
 (1287.7957912650593, 6),
 (1274.7740976345574, 6),
 (1372.5891592169887, 6),
 (1159.7279853482885, 6),
 (209.77130404323657, 6),
 (171.96802028284213, 6),
 (1377.510072558455, 6),
 (1214.1091384220779, 6),
 (1222.5048057165257, 6),
 (179.42407865166817, 6),
 (

In [110]:
len(mycounter)

1286

In [101]:
distance_between_beacons(sen[0])

[1317.0045558007762,
 2367.6634051317346,
 139.38794782907166,
 1066.2809198330428,
 1549.5760710594366,
 1510.3820708681628,
 1136.87510307861,
 2436.6914043431925,
 1085.624244386611,
 1002.1207512071587,
 1956.4631353542034,
 1509.0536769777277,
 1792.8792485831275,
 145.74635501445653,
 1079.7110724633696,
 2095.407120346784,
 2329.8894823574788,
 1840.0486406614364,
 1339.9085789709684,
 1074.4924383168081,
 1070.6115075040059,
 1951.5926829131124,
 1939.0443522519024,
 1308.5832033157083,
 1317.0045558007762,
 1869.3145802673234,
 1210.3189662233672,
 1385.0321295912236,
 1054.4235391909647,
 934.4180006827779,
 1553.0312295636556,
 1940.5658968455568,
 1551.4435213696952,
 1389.008639281988,
 1251.0831307311278,
 1028.3491624929734,
 2110.1708461638837,
 1299.1970597257368,
 1362.9075537247566,
 1410.487149888293,
 1858.4708768232017,
 2139.381686375762,
 122.53162856993292,
 934.8481160060173,
 1247.714710981641,
 1251.783927041724,
 2244.3123668509247,
 94.45104552094699,
 236

In [89]:
beste = None
mostmatched = 0
for e,cand in enumerate(transform(sen[1])):
    (sen[0] - sen[1])
    

array([[ 686,  422,  578],
       [ 605,  423,  415],
       [ 515,  917, -361],
       [-336,  658,  858],
       [  95,  138,   22],
       [-476,  619,  847],
       [-340, -569, -846],
       [ 567, -361,  727],
       [-460,  603, -452],
       [ 669, -402,  600],
       [ 729,  430,  532],
       [-500, -761,  534],
       [-322,  571,  750],
       [-466, -666, -811],
       [-429, -592,  574],
       [-355,  545, -477],
       [ 703, -491, -529],
       [-328, -685,  520],
       [ 413,  935, -424],
       [-391,  539, -444],
       [ 586, -435,  557],
       [-364, -763, -893],
       [ 807, -499, -711],
       [ 755, -354, -619],
       [ 553,  889, -390]])

In [70]:
len(dir)

32

In [71]:
dir

[(-1, -1, -1, 0),
 (1, -1, -1, 0),
 (-1, 1, -1, 0),
 (1, 1, -1, 0),
 (-1, -1, 1, 0),
 (1, -1, 1, 0),
 (-1, 1, 1, 0),
 (1, 1, 1, 0),
 (-1, -1, -1, 90),
 (1, -1, -1, 90),
 (-1, 1, -1, 90),
 (1, 1, -1, 90),
 (-1, -1, 1, 90),
 (1, -1, 1, 90),
 (-1, 1, 1, 90),
 (1, 1, 1, 90),
 (-1, -1, -1, 180),
 (1, -1, -1, 180),
 (-1, 1, -1, 180),
 (1, 1, -1, 180),
 (-1, -1, 1, 180),
 (1, -1, 1, 180),
 (-1, 1, 1, 180),
 (1, 1, 1, 180),
 (-1, -1, -1, 270),
 (1, -1, -1, 270),
 (-1, 1, -1, 270),
 (1, 1, -1, 270),
 (-1, -1, 1, 270),
 (1, -1, 1, 270),
 (-1, 1, 1, 270),
 (1, 1, 1, 270)]

In [140]:
from copy import deepcopy
from itertools import combinations
import math 

def rotations():
    """Generate all possible rotation functions"""
    vectors = [
        (1, 0, 0),
        (-1, 0, 0),
        (0, 1, 0),
        (0, -1, 0),
        (0, 0, 1),
        (0, 0, -1),
    ]
    vectors = list(map(np.array, vectors))
    for vi in vectors:
        for vj in vectors:
            if vi.dot(vj) == 0:
                vk = np.cross(vi, vj)
                yield lambda x: np.matmul(x, np.array([vi, vj, vk]))


def fit(scanners, hashes, i, j, v):
    """Find the correct rotation/translation to make the jth scanner map fit the ith"""
    s1, s2 = scanners[i], scanners[j]
    for rot in rotations():
        s2t = rot(s2)
        p = hashes[i][v][0]
        for q in hashes[j][v]:
            diff = s1[p, :] - s2t[q, :]
            if len((b := set(map(tuple, s2t + diff))) & set(map(tuple, s1))) >= 12:
                return diff, b, rot


def map_hash(coords):
    """
    Generate a hashset of sorted absolute coordinate differences
    between pairs of points
    """
    s = {
        tuple(sorted(map(abs, coords[i, :] - coords[j, :]))): (i, j)
        for i, j in combinations(range(len(coords)), 2)
    }
    return s


def match(scanners, hashes):
    """Figure out which pairs of scanner aps have sufficient overlap"""
    for i, j in combinations(range(len(hashes)), 2):
        if len(m := set(hashes[i]) & set(hashes[j])) >= math.comb(12, 2):
            yield i, j, next(iter(m))


def solve(scanners):
    """Given a list of scanner maps, return list of positions and set of beacons"""
    scanners = deepcopy(scanners)
    positions = {0: (0, 0, 0)}
    hashes = list(map(map_hash, scanners))
    beacons = set(map(tuple, scanners[0]))
    while len(positions) < len(scanners):
        for i, j, v in match(scanners, hashes):
            if not (i in positions) ^ (j in positions):
                continue
            elif j in positions:
                i, j = j, i
            positions[j], new_beacons, rot = fit(scanners, hashes, i, j, v)
            scanners[j] = rot(scanners[j]) + positions[j]
            beacons |= new_beacons
            
    return [positions[i] for i in range(len(scanners))], beacons

# f = open('input.txt','r')
# data = f.read()
# scanners = data.split("\n\n")
# scanners = [x.split("---\n")[-1].split("\n") for x in scanners]
# scanners = [np.array([list(map(int, y.split(","))) for y in x]) for x in scanners]
f = open('input.txt')
lines = f.read()
sensors = [[list(map(int,x.split(','))) for x in l.split('\n')[1:-1]] for l in lines.replace('\n\n','\n').split('--- scanner')][1:]
scanners = []
for s,sensor in enumerate(sensors):
    scanners.append(np.asarray(sensor))
    
positions, beacons = solve(scanners)
print(len(beacons))
print(max(np.abs(x - y).sum() for x, y in combinations(positions, 2)))

353
10832


In [130]:
len(sensors)

5

In [138]:
math.comb(12,2)

66

In [ ]:
map